# Merging ICESAT, MODIS, and LANDSAT Satellite Data

In [263]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import swifter

/Users/patrickgeitner/miniforge3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Merging MODIS and ICESAT Data
### Great Barrier Reef

In [2]:
icesat_gb = pd.read_pickle('./files/ICESAT2/icesat_data_GB_filtered_heights.pkl')
icesat_gb = gpd.GeoDataFrame(icesat_gb, geometry=gpd.points_from_xy(icesat_gb.lons, icesat_gb.lats))

In [52]:
modis_gb = pd.read_pickle('./files/MODIS/modis_GB.pkl')
modis_gb = gpd.GeoDataFrame(modis_gb, geometry=gpd.points_from_xy(modis_gb.longitude, modis_gb.latitude))
modis_gb['datetime'] = modis_gb.time.apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
modis_gb['datetime'] = pd.DatetimeIndex(modis_gb['datetime'])

In [53]:
icesat_modis_gb = gpd.sjoin_nearest(icesat_gb, modis_gb, how='inner', lsuffix='icesat', rsuffix='modis', max_distance=.001, distance_col='distance')

In [54]:
icesat_modis_gb = icesat_modis_gb[['lats', 'lons', 'datetime_modis', 'datetime_icesat', 'heights', 'chlor_a', 'distance']]
icesat_modis_gb['distance_meters_icesat_modis'] = icesat_modis_gb.distance * 111000
icesat_modis_gb['time_diff_icesat_modis'] = np.abs(icesat_modis_gb.datetime_modis - icesat_modis_gb.datetime_icesat)
print(icesat_modis_gb.shape)
icesat_modis_gb.head()

(12058, 9)


,lats,lons,datetime_modis,datetime_icesat,heights,chlor_a,distance,distance_meters_icesat_modis,time_diff_icesat_modis
8183806,-14.328735,144.852555,2020-08-14 20:55:00,2018-10-19 05:43:13,-14.017767,0.399796,0.000992,110.095998,665 days 15:11:47
8183807,-14.328734,144.852554,2020-08-14 20:55:00,2018-10-19 05:43:13,1.471302,0.399796,0.000991,110.053046,665 days 15:11:47
8183818,-14.328704,144.852551,2020-08-14 20:55:00,2018-10-19 05:43:13,-20.322205,0.399796,0.000976,108.289021,665 days 15:11:47
8183839,-14.328659,144.852547,2020-08-14 20:55:00,2018-10-19 05:43:13,-21.377756,0.399796,0.000954,105.884897,665 days 15:11:47
8183844,-14.328646,144.852545,2020-08-14 20:55:00,2018-10-19 05:43:13,-8.550291,0.399796,0.000948,105.201120,665 days 15:11:47


## Caribbean

In [6]:
icesat_car = pd.read_pickle('./files/ICESAT2/icesat_data_CAR_filtered_height.pkl')
icesat_car = gpd.GeoDataFrame(icesat_car, geometry=gpd.points_from_xy(icesat_car.lons, icesat_car.lats))

In [55]:
modis_car = pd.read_pickle('./files/MODIS/modis_CAR.pkl')
modis_car = gpd.GeoDataFrame(modis_car, geometry=gpd.points_from_xy(modis_car.longitude, modis_car.latitude))
modis_car['datetime'] = modis_car.time.apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
modis_car['datetime'] = pd.DatetimeIndex(modis_car['datetime'])

In [56]:
icesat_modis_car = gpd.sjoin_nearest(icesat_car, modis_car, how='inner', lsuffix='icesat', rsuffix='modis', max_distance=.001, distance_col='distance')

In [57]:
icesat_modis_car = icesat_modis_car[['lats', 'lons', 'datetime_modis', 'datetime_icesat', 'heights', 'chlor_a', 'distance']]
icesat_modis_car['distance_meters_icesat_modis'] = icesat_modis_car.distance * 111000
icesat_modis_car['time_diff_icesat_modis'] = np.abs(icesat_modis_car.datetime_modis - icesat_modis_car.datetime_icesat)
print(icesat_modis_car.shape)
icesat_modis_car.head()

(843793, 9)


,lats,lons,datetime_modis,datetime_icesat,heights,chlor_a,distance,distance_meters_icesat_modis,time_diff_icesat_modis
538789,25.827547,-74.245576,2018-09-27 20:10:00,2018-10-14 08:35:34,-44.081188,0.029181,0.001000,110.965367,16 days 12:25:34
538790,25.827547,-74.245576,2018-09-27 20:10:00,2018-10-14 08:35:34,-43.814438,0.029181,0.001000,110.963332,16 days 12:25:34
538791,25.827541,-74.245577,2018-09-27 20:10:00,2018-10-14 08:35:34,-43.801620,0.029181,0.000993,110.258293,16 days 12:25:34
538792,25.827535,-74.245577,2018-09-27 20:10:00,2018-10-14 08:35:34,-44.236015,0.029181,0.000987,109.547328,16 days 12:25:34
538793,25.827515,-74.245580,2018-09-27 20:10:00,2018-10-14 08:35:34,-43.847576,0.029181,0.000968,107.401833,16 days 12:25:34


In [58]:
icesat_modis = pd.concat([icesat_modis_gb, icesat_modis_car])
icesat_modis.to_pickle('./files/icesat_modis.pkl')

## Adding Landsat Data from Google Earth Engine

In [296]:
import ee
import pandas as pd
ee.Authenticate(quiet=True)
ee.Initialize()

Paste the following address into a web browser:

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=WexqNmLw04AmJYa1nXDRQKg29SPre5FiM2mdEcP1lDA&tc=aeaqR7ApCRjZNMQ4JBeSDm6eNKn5tCFEMQUua0amdHw&cc=2j5uQlq1-fio2_AR7PK57LRistrUxE42g_D4f22b0I8

On the web page, please authorize access to your Earth Engine account and copy the authentication code. Next authenticate with the following command:

    earthengine authenticate --code-verifier=WexqNmLw04AmJYa1nXDRQKg29SPre5FiM2mdEcP1lDA:0F2EcRRBuTi3Y9CiDregeBXR6iAEmNyvePXiJKcpN_o:nNtjQD-QzUl0PZsA6d3rCR9TmOjNLPMQI8CzVzFhKkA --authorization-code=PLACE_AUTH_CODE_HERE

Enter verification code: 4/1ARtbsJqu6WPQF9r2aMnuRRzUTmfm4ORJeGsGmOB-YHAIPS9MjQgHrx7nLJY

Successfully saved authorization token.


In [297]:
icesat_modis['query_start_date'] = icesat_modis['datetime_modis'] - datetime.timedelta(days=15)
icesat_modis['query_end_date'] = icesat_modis['datetime_modis'] + datetime.timedelta(days=15)

icesat_modis['start_day'] = pd.DatetimeIndex(icesat_modis['query_start_date']).day.astype(str).str.zfill(2)
icesat_modis['end_day'] = pd.DatetimeIndex(icesat_modis['query_end_date']).day.astype(str).str.zfill(2)

icesat_modis['start_month'] = pd.DatetimeIndex(icesat_modis['query_start_date']).month.astype(str).str.zfill(2)
icesat_modis['end_month'] = pd.DatetimeIndex(icesat_modis['query_end_date']).month.astype(str).str.zfill(2)

icesat_modis['start_year'] = pd.DatetimeIndex(icesat_modis['query_start_date']).year.astype(str)
icesat_modis['end_year'] = pd.DatetimeIndex(icesat_modis['query_end_date']).year.astype(str)

In [305]:
from ee import EEException

def getLandsatData(row):
    try:
        landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'])
        image = landsat.filterBounds(geometry=ee.Geometry.Point([row['lons'], row['lats']])).filterDate(f"{row['start_year']}-{row['start_month']}-{row['start_day']}", f"{row['end_year']}-{row['end_month']}-{row['end_day']}")
        data = image.getRegion(geometry=ee.Geometry.Point([row['lons'], row['lats']]), scale=100).getInfo()
        df = pd.DataFrame(data[1:], columns = data[0])
        df['datetime'] = df.time.apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
        if df.shape[0] > 1:
            time_deltas = {}
            for i in range(df.shape[0]):
                time_deltas[i] = np.abs((df['datetime'].values[i] - row['datetime_modis']))
            ind = min(time_deltas, key=time_deltas.get)
            row['SR_B1'] = df.iloc[ind,]['SR_B1']
            row['SR_B2'] = df.iloc[ind,]['SR_B2']
            row['SR_B3'] = df.iloc[ind,]['SR_B3']
            row['SR_B4'] = df.iloc[ind,]['SR_B4']
            row['SR_B5'] = df.iloc[ind,]['SR_B5']
            row['SR_B7'] = df.iloc[ind,]['SR_B7']
            row['QA_PIXEL'] = df.iloc[ind,]['QA_PIXEL']
            row['datetime_landsat'] = df.iloc[ind,]['datetime']
        elif df.shape[0]==1:
            row['SR_B1'] = df.iloc[0,]['SR_B1']
            row['SR_B2'] = df.iloc[0,]['SR_B2']
            row['SR_B3'] = df.iloc[0,]['SR_B3']
            row['SR_B4'] = df.iloc[0,]['SR_B4']
            row['SR_B5'] = df.iloc[0,]['SR_B5']
            row['SR_B7'] = df.iloc[0,]['SR_B7']
            row['QA_PIXEL'] = df.iloc[0,]['QA_PIXEL']
            row['datetime_landsat'] = df.iloc[0,]['datetime']
        elif df.shape[0]==0:
            row['SR_B1'] = np.nan
            row['SR_B2'] = np.nan
            row['SR_B3'] = np.nan
            row['SR_B4'] = np.nan
            row['SR_B5'] = np.nan
            row['SR_B7'] = np.nan
            row['QA_PIXEL'] = np.nan
            row['datetime_landsat'] = np.nan
        return row
    except EEException:
        row['SR_B1'] = np.nan
        row['SR_B2'] = np.nan
        row['SR_B3'] = np.nan
        row['SR_B4'] = np.nan
        row['SR_B5'] = np.nan
        row['SR_B7'] = np.nan
        row['QA_PIXEL'] = np.nan
        row['datetime_landsat'] = np.nan
        return row
        

In [306]:
icesat_modis = icesat_modis.reset_index(drop=True)
icesat_modis = icesat_modis.iloc[:250000,]

In [307]:
icesat_modis_landsat = icesat_modis.swifter.apply(getLandsatData, axis=1)

Pandas Apply:   0%|          | 0/250000 [00:00<?, ?it/s]

In [308]:
icesat_modis_landsat = icesat_modis_landsat.dropna()
icesat_modis_landsat.shape

(123441, 28)

In [309]:
icesat_modis_landsat.to_pickle('./files/merged_sat_data_first250k.pkl')

In [310]:
icesat_modis_landsat.head()

,lats,lons,datetime_modis,datetime_icesat,heights,chlor_a,distance,distance_meters_icesat_modis,time_diff_icesat_modis,day,...,start_year,end_year,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B7,QA_PIXEL,datetime_landsat
0,-14.328735,144.852555,2020-08-14 20:55:00,2018-10-19 05:43:13,-14.017767,0.399796,0.000992,110.095998,665 days 15:11:47,14,...,2020,2020,8659.0,9541.0,10982.0,9817.0,7178.0,7573.0,21952.0,2020-08-12 20:27:57.749
1,-14.328734,144.852554,2020-08-14 20:55:00,2018-10-19 05:43:13,1.471302,0.399796,0.000991,110.053046,665 days 15:11:47,14,...,2020,2020,8659.0,9541.0,10982.0,9817.0,7178.0,7573.0,21952.0,2020-08-12 20:27:57.749
2,-14.328704,144.852551,2020-08-14 20:55:00,2018-10-19 05:43:13,-20.322205,0.399796,0.000976,108.289021,665 days 15:11:47,14,...,2020,2020,8659.0,9541.0,10982.0,9817.0,7178.0,7573.0,21952.0,2020-08-12 20:27:57.749
3,-14.328659,144.852547,2020-08-14 20:55:00,2018-10-19 05:43:13,-21.377756,0.399796,0.000954,105.884897,665 days 15:11:47,14,...,2020,2020,8659.0,9541.0,10982.0,9817.0,7178.0,7573.0,21952.0,2020-08-12 20:27:57.749
4,-14.328646,144.852545,2020-08-14 20:55:00,2018-10-19 05:43:13,-8.550291,0.399796,0.000948,105.201120,665 days 15:11:47,14,...,2020,2020,8659.0,9541.0,10982.0,9817.0,7178.0,7573.0,21952.0,2020-08-12 20:27:57.749


In [311]:
icesat_modis_landsat['end_year'].unique()

array(['2020', '2018', '2019', '2021'], dtype=object)

In [312]:
icesat_modis_landsat.columns

Index(['lats', 'lons', 'datetime_modis', 'datetime_icesat', 'heights',
       'chlor_a', 'distance', 'distance_meters_icesat_modis',
       'time_diff_icesat_modis', 'day', 'month', 'year', 'end_day',
       'query_start_date', 'query_end_date', 'start_day', 'start_month',
       'end_month', 'start_year', 'end_year', 'SR_B1', 'SR_B2', 'SR_B3',
       'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL', 'datetime_landsat'],
      dtype='object')